In [1]:
using CSV, DataFrames

In [2]:
@show primAuthor = "Tanasa"
fileOwnPapers = "RT_JL_savedrecs_mine.txt"
fileCitations = "RT_JL_savedrecs.txt"
fileOwnCSV    = "RT_JL_MINE_test.csv"
fileOwnTXT    = "RT_JL_total.txt"
fileOwnTXT5   = "RT_JL_total5.txt"
fileOwnUAIC   = "RT_JL_UAIC1.txt"
#root = dirname(@__FILE__)
root = dirname("/home/lali/TITAN-ROG-sync/julia/ISI/")
@show inputAIS = joinpath(root, "AI_factor_full_1997-2024_plus.txt")
@show inputISI = joinpath(root, "IF_factor_full_1997-2024_plus.txt")

@show inputMINE = joinpath(root, fileOwnPapers);
@show inputCIT = joinpath(root, fileCitations);

primAuthor = "Tanasa" = "Tanasa"
inputAIS = joinpath(root, "AI_factor_full_1997-2024_plus.txt") = "/home/lali/TITAN-ROG-sync/julia/ISI/AI_factor_full_1997-2024_plus.txt"
inputISI = joinpath(root, "IF_factor_full_1997-2024_plus.txt") = "/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2024_plus.txt"
inputMINE = joinpath(root, fileOwnPapers) = "/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_savedrecs_mine.txt"
inputCIT = joinpath(root, fileCitations) = "/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_savedrecs.txt"


In [3]:
@show AIFirstYear = 1997;
@show AILastYear = 2024;

@show ANini  = 1997;
@show ANini5 = 2021;
@show ANfin  = 2024;

AIFirstYear = 1997 = 1997
AILastYear = 2024 = 2024
ANini = 1997 = 1997
ANini5 = 2021 = 2021
ANfin = 2024 = 2024


In [4]:
AIS = CSV.read(inputAIS, DataFrame; header=true, delim='\t');

In [5]:
ISI = CSV.read(inputISI, DataFrame; header=true, delim='\t');

In [6]:
#@show AIS[8157, [1, 3, 5, 6,    29, 30]];
#@show ISI[8157, [1, 3, 5, 6,    29, 30]];
AIS = dropmissing(AIS, :Journal)
ISI = dropmissing(ISI, :Journal);

In [7]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true, delim='\t');
MINE = MINEraw[:, [2, 9, 25, 33, 44, 47, 48, 54, 55, 56, 57]];
#                 AU TI  RP  NR  J9  PY  VL  BP  EP  AR  DI

sort!(MINE, [:PY], rev = true)
eltype.(eachcol(MINE));

MINEraw=nothing;

In [8]:
@show MINE[1, [:AU, :J9, :PY]];

MINE[1, [:AU, :J9, :PY]] = DataFrameRow
 Row │ AU                                 J9            PY
     │ String                             String31      Int64
─────┼────────────────────────────────────────────────────────
   1 │ Andriesei, A; Plesca, D; Capu, R…  ROM REP PHYS   2023


In [9]:
#   PT	AU	BA	BE	GP	AF	BF	CA	TI	SO	SE	BS	LA	DT	CT	CY	CL	SP	HO	DE	ID	AB	C1	
#   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  

#   C3	RP	EM	RI	OI	FU	FP	FX	CR	NR	TC	Z9	U1	U2	PU	PI	PA	SN	EI	BN	J9	JI	PD	
#   24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  

#   PY	VL	IS	PN	SU	SI	MA	BP	EP	AR	DI	DL	D2	EA	PG	WC	WE	SC	GA	PM	OA	HC	HP	DA	UT
#   47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71

#   ######  2   9   32  44  46  47  48  54  55  56  57
#   ######  AU  TI  CR  J9  PD  PY  VL  BP  EP  AR  DI

In [10]:
MINECITEraw = CSV.read(inputCIT, DataFrame; rows_to_check = 1000, header=true, delim='\t');
MINECITE = MINECITEraw[:, [2, 9, 32, 44, 46, 47, 48, 54, 55, 56, 57]];
#                         AU TI  CR  J9  PD  PY  VL  BP  EP  AR  DI
MINECITEraw=nothing;

In [11]:
# @show MINECITE[1, [:AU, :J9, :PY]];

###########################################################  
####################  Finished loading   ########################## 
###########################################################

In [12]:
rename!(MINE,:"J9" => :"Journal")
rename!(MINECITE,:"J9" => :"Journal");

insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);
insertcols!(MINE, :CT => 0.0);
### 5 ani
insertcols!(MINE, :PRIM5 => 0);
insertcols!(MINE, :AINEFF5 => 0.0);
insertcols!(MINE, :IFNA5 => 0.0);
insertcols!(MINE, :TIMESC5 => 0);
insertcols!(MINE, :CT5 => 0.0);
### UAIC
insertcols!(MINE, :UAIC1 => 0.0);
insertcols!(MINE, :UAIC9 => 0.0);

Găsim numarul de autori și prim autor

In [13]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    #println( titlecase(MINE[row_index, :AU]))
    #println(findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1])
    if findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1] == 1
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini5)
            MINE[row_index, :PRIM5] = 1
        end
    elseif !ismissing(lucrare[:RP])
        if occursin(primAuthor, lucrare[:RP])
            MINE[row_index, :PRIM] = 1
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini5)
                MINE[row_index, :PRIM5] = 1
            end
        end
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

În caz că lipsește un publication year:

In [14]:
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

Row,AU,PY,VL,BP,NA,PRIM,AI,AINEFF,IF
,String,Int64,Int64,Int64?,Int64,Int64,Float64,Float64,Float64


Subseturi (?):

In [15]:
#AISsubset = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#size(AISsubset, 1);

In [16]:
#ISIsubset = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), ISI)
#size(ISIsubset, 1);

Căutare MINE

In [17]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
    #for AISrow in eachrow(AISsubset)
        if( lucrare[:Journal] == AISrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini5)
                MINE[row_index, :AINEFF5] = MINE[row_index, :AINEFF]
            end
        end
    end
    for ISIrow in eachrow(ISI)
    #for ISIrow in eachrow(ISIsubset)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini5)
                MINE[row_index, :IFNA5] = MINE[row_index, :IFNA]
                MINE[row_index, :UAIC1] = (25 + 60*MINE[row_index, :IF]) / MINE[row_index, :NA]
            end
        end
    end
end

Rezultate MINE:

In [18]:
I = sum(MINE[!, :AINEFF]);
P = sum(MINE[!, :PRIM] .* MINE[!, :AI]);
I5 = sum(MINE[!, :AINEFF5]);
P5 = sum(MINE[!, :PRIM5] .* MINE[!, :AI]);
UAIC_1_1 = sum(MINE[!, :UAIC1]);

I, P, I5, P5, UAIC_1_1

(6.809322727272727, 9.686, 2.0372045454545455, 0.0, 429.4828354978355)

###########################################################  
####################  Citări   ################################## 
###########################################################

Verificare prezență autocitări

In [19]:
# eliminare autocitari
MINECITE = filter(row -> !(occursin(primAuthor, row.AU)),  MINECITE);

In [20]:
# print autocitari
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
    if( occursin(primAuthor, lucrareCIT.AU) )
        println(row_indexCIT, "  ", lucrareCIT.AU)
    end
end

Unele citări fără PY (early access) - CHECK!!!

In [21]:
# print citări fără PY (early access) și set to 2024
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
    if( ismissing(lucrareCIT.PY) )
        println(row_indexCIT, "  ", lucrareCIT.AU)
        lucrareCIT.PY = 2024
    end
end

In [22]:
CITES = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES, [0])
end
CITES_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES_IFs, [0.0])
end

In [23]:
CITES5 = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES5, [0])
end
CITES5_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES5_IFs, [0.0])
end

In [24]:
for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
    calcul_cit_UAIC = 0.0
    if ( ismissing(lucrareMINE[:DI]) )
        #println("Skip paper $row_indexMINE noDOI")
        continue
    end
    #println("Analyzing paper ", lucrareMINE[:DI])
    for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
        if ( occursin(lucrareMINE[:DI], lucrareCIT[:CR]) )
            #println("Paper $row_indexMINE cited by $row_indexCIT")
            ISI_value = 0.0
            push!(CITES[row_indexMINE], row_indexCIT)
            push!(CITES_IFs[row_indexMINE], ISI_value)
            if (( !ismissing(lucrareCIT[:PY]) ) && ( !ismissing(lucrareCIT[:Journal]) ))
                for ISIrow in eachrow(ISI)
                    if( lucrareCIT[:Journal] == ISIrow[:Journal] )
                        an_publicare_citare = lucrareCIT[:PY]
                        if (an_publicare_citare > AILastYear)
                            an_publicare_citare = AILastYear
                        end
                        if (an_publicare_citare < AIFirstYear)
                            an_publicare_citare = AIFirstYear
                        end
                        ISI_value = ISIrow[string(an_publicare_citare)]
                        push!(CITES_IFs[row_indexMINE], ISI_value)
                        #CITES_IFs[row_indexMINE, size(CITES_IFs[row_indexMINE])] = ISI_value
                        break
                    end
                end
                if (lucrareCIT[:PY] <= ANfin) && (lucrareCIT[:PY] >= ANini5)
                    push!(CITES5[row_indexMINE], row_indexCIT)
                    push!(CITES5_IFs[row_indexMINE], ISI_value)
                    calcul_cit_UAIC = calcul_cit_UAIC + (10 + 20 * ISI_value) / lucrareMINE[:NA]
                end
            end
        end
    end
    MINE[row_indexMINE, :TIMESC] = size(CITES[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT] = MINE[row_indexMINE, :TIMESC] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :TIMESC5] = size(CITES5[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT5] = MINE[row_indexMINE, :TIMESC5] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :UAIC9] = calcul_cit_UAIC
end;

In [25]:
#@show CITES[27]
#size(CITES5_IFs[2])

In [26]:
C = sum(MINE[!, :CT])
C5 = sum(MINE[!, :CT5])
UAIC_1_9 = sum(MINE[!, :UAIC9])

C, C5, UAIC_1_9

(92.47229437229437, 22.27619047619048, 2134.89597979798)

###########################################################  
####################  SAVE   ################################## 
###########################################################

In [27]:
using Printf

In [28]:
@show outputMINEUAICtxt = joinpath(root, fileOwnUAIC)
@show outputMINEtxt  = joinpath(root, fileOwnTXT)
@show outputMINEtxt5 = joinpath(root, fileOwnTXT5)

@show outputMINEcsv = joinpath(root, fileOwnCSV)

CSV.write(outputMINEcsv, MINE)

outputMINEUAICtxt = joinpath(root, fileOwnUAIC) = "/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_UAIC1.txt"
outputMINEtxt = joinpath(root, fileOwnTXT) = "/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_total.txt"
outputMINEtxt5 = joinpath(root, fileOwnTXT5) = "/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_total5.txt"
outputMINEcsv = joinpath(root, fileOwnCSV) = "/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_MINE_test.csv"


"/home/lali/TITAN-ROG-sync/julia/ISI/RT_JL_MINE_test.csv"

In [29]:
#ENV["COLUMNS"]=500
#MINE[1, filter(x -> !(x in ["CR","RP"]), names(MINE))]

In [30]:
open(outputMINEUAICtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "nr.autori: $(lucrareMINE.NA) \t\t IF: $(lucrareMINE.IF) \t\t UAIC I.1(ISI): $(@sprintf("%8.4f", lucrareMINE.UAIC1)) \t\t nr.citari: $(lucrareMINE.TIMESC5) \t\t UAIC I.9(Citari): $(@sprintf("%9.4f", lucrareMINE.UAIC9)) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '-'^400, '\n')
    write(file, '-'^400, '\n')
    write(file, "Total criteriul I.1(UAIC) = $(@sprintf("%9.4f", UAIC_1_1))\t\t\tTotal criteriul I.9(UAIC) = $(@sprintf("%9.4f", UAIC_1_9))")
end

77

In [31]:
open(outputMINEtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            #write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "nr.autori: $(lucrareMINE.NA) \t\t IF: $(lucrareMINE.IF) \t\t nr.citari: $(lucrareMINE.TIMESC5) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '-'^400, '\n')
    write(file, '-'^400, '\n')
    #write(file, "Total criteriul I.1(UAIC) = $(@sprintf("%9.4f", UAIC_1_1))\t\t\tTotal criteriul I.9(UAIC) = $(@sprintf("%9.4f", UAIC_1_9))")
    write(file, "P = $(@sprintf("%9.4f", P)) \t\t\t I = $(@sprintf("%9.4f", I)) \t\t\t C = $(@sprintf("%9.4f", C)) \n")
end

51

In [32]:
open(outputMINEtxt5, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            #write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "nr.autori: $(lucrareMINE.NA) \t\t IF: $(lucrareMINE.IF) \t\t nr.citari: $(lucrareMINE.TIMESC5) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '-'^400, '\n')
    write(file, '-'^400, '\n')
    #write(file, "Total criteriul I.1(UAIC) = $(@sprintf("%9.4f", UAIC_1_1))\t\t\tTotal criteriul I.9(UAIC) = $(@sprintf("%9.4f", UAIC_1_9))")
    write(file, "P = $(@sprintf("%9.4f", P5)) \t\t\t I = $(@sprintf("%9.4f", I5)) \t\t\t C = $(@sprintf("%9.4f", C5)) \n")
end

51